# Sentiment Data Test Model Vector Space Model (SMALL)

## Load data

In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
moviereaction = pd.read_json('data/MovieReactionDS_small.json')

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from gensim.parsing.preprocessing import preprocess_string
from sklearn import utils
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tqdm import tqdm
import multiprocessing

In [6]:
class Doc2VecTransformer(BaseEstimator): 
    def __init__(self, vector_size=100, learning_rate=0.02, epochs=20):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self._model = None
        self.vector_size = vector_size
        self.workers = multiprocessing.cpu_count() 

    def fit(self, df_x, df_y=None):
        tagged_x = [TaggedDocument(preprocess_string(row['input']), [index]) for index, row in df_x.iterrows()]
        model = Doc2Vec(documents=tagged_x, vector_size=self.vector_size, workers=self.workers) 

        for epoch in range(self.epochs):
            model.train(utils.shuffle([x for x in tqdm(tagged_x)]), 
        total_examples=len(tagged_x), epochs=1)
        model.alpha -= self.learning_rate
        model.min_alpha = model.alpha 

        self._model = model
        return self 
    
    def transform(self, df_x):
        return np.asmatrix(np.array([self._model.infer_vector(preprocess_string(row['input'])) for index, row in df_x.iterrows()]))

In [7]:
def train_and_build_model():
    all_reviews_df = moviereaction
    train_x_df, test_x_df, train_y_df, test_y_df = train_test_split(all_reviews_df[['input']], all_reviews_df[['output']])

    pl = Pipeline(steps=[('doc2vec', Doc2VecTransformer(vector_size=220)),('pca', PCA(n_components=100)),('logistic', LogisticRegression())])
    pl.fit(train_x_df[['input']], train_y_df[['output']])
    predictions_y = pl.predict(test_x_df[['input']])
    print('Accuracy: ', accuracy_score(y_true=test_y_df[['output']], y_pred=predictions_y))
    print('\n')
    print('Confusion Matrix:', '\n',confusion_matrix(y_true=test_y_df[['output']], y_pred=predictions_y))
    print('Classification Report:', '\n',classification_report(y_true=test_y_df[['output']], y_pred=predictions_y))

In [8]:
def train_short_range_grid_search():
    all_reviews_df = moviereaction
    train_x_df, test_x_df, train_y_df, test_y_df = train_test_split(all_reviews_df[['input']], all_reviews_df[['output']])

    pl = Pipeline(steps=[('doc2vec', Doc2VecTransformer()),
    ('pca', PCA()),
    ('logistic', LogisticRegression())])

    param_grid = {
    'doc2vec__vector_size': [200, 220, 250],
    'pca__n_components': [50, 75, 100]
    }
    gs_cv = GridSearchCV(estimator=pl, param_grid=param_grid, cv=3, n_jobs=-1,
    scoring="accuracy")
    gs_cv.fit(train_x_df[['input']], train_y_df[['output']])

    print("Best parameter (CV score=%0.3f):" % gs_cv.best_score_)
    print(gs_cv.best_params_)
    predictions_y = gs_cv.predict(test_x_df[['input']])
    print('Accuracy: ', accuracy_score(y_true=test_y_df[['output']], y_pred=predictions_y))
    print('\n')
    print('Confusion Matrix:', '\n',confusion_matrix(y_true=test_y_df[['output']], y_pred=predictions_y))
    print('Classification Report:', '\n',classification_report(y_true=test_y_df[['output']], y_pred=predictions_y))

In [9]:
def train_long_range_grid_search():
    all_reviews_df = moviereaction
    train_x_df, test_x_df, train_y_df, test_y_df = train_test_split(all_reviews_df[['input']], all_reviews_df[['output']]) 
    
    pl = Pipeline(steps=[('doc2vec', Doc2VecTransformer()),('pca', PCA()),('logistic', LogisticRegression())]) 
    
    param_grid = {
    'doc2vec__vector_size': [x for x in range(100, 250)],
    'pca__n_components': [x for x in range(1, 50)]
    }
    gs_cv = GridSearchCV(estimator=pl, param_grid=param_grid, cv=5, n_jobs=-1,
    scoring="accuracy")
    gs_cv.fit(train_x_df[['input']], train_y_df[['output']]) 
    
    print("Best parameter (CV score=%0.3f):" % gs_cv.best_score_)
    print(gs_cv.best_params_)
    predictions_y = gs_cv.predict(test_x_df[['input']])
    print('Accuracy: ', accuracy_score(y_true=test_y_df[['output']], y_pred=predictions_y))
    print('\n')
    print('Confusion Matrix:', '\n',confusion_matrix(y_true=test_y_df[['output']], y_pred=predictions_y))
    print('Classification Report:', '\n',classification_report(y_true=test_y_df[['output']], y_pred=predictions_y))

In [10]:
train_short_range_grid_search()

c:\Users\Romando\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]
  warnings.warn(
100%|██████████| 21/21 [00:00<?, ?it/s]


ValueError: n_components=50 must be between 0 and min(n_samples, n_features)=21 with svd_solver='full'